<a href="https://colab.research.google.com/github/Jairex22/Google-AI-Burger-picture/blob/main/Te_damos_la_bienvenida_a_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
# Import required library
import numpy as np

# Sigmoid activation function and its derivative
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Sample input data (4 examples, 2 features each)
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

# Target outputs (XOR pattern for challenge)
y = np.array([[0], [1], [1], [0]])

# Set random seed for reproducibility
np.random.seed(42)

# Initialize weights and biases randomly
input_neurons = 2
hidden_neurons = 4
output_neurons = 1

# Weight matrix from input to hidden layer
W1 = np.random.uniform(size=(input_neurons, hidden_neurons))
# Bias for hidden layer
b1 = np.random.uniform(size=(1, hidden_neurons))

# Weight matrix from hidden to output layer
W2 = np.random.uniform(size=(hidden_neurons, output_neurons))
# Bias for output layer
b2 = np.random.uniform(size=(1, output_neurons))

# Training loop
epochs = 10000
learning_rate = 0.1

for epoch in range(epochs):
    # Forward Propagation
    hidden_input = np.dot(X, W1) + b1
    hidden_output = sigmoid(hidden_input)

    final_input = np.dot(hidden_output, W2) + b2
    final_output = sigmoid(final_input)

    # Calculate the error (loss)
    error = y - final_output

    # Backpropagation
    d_output = error * sigmoid_derivative(final_output)
    error_hidden = d_output.dot(W2.T)
    d_hidden = error_hidden * sigmoid_derivative(hidden_output)

    # Update weights and biases
    W2 += hidden_output.T.dot(d_output) * learning_rate
    b2 += np.sum(d_output, axis=0, keepdims=True) * learning_rate
    W1 += X.T.dot(d_hidden) * learning_rate
    b1 += np.sum(d_hidden, axis=0, keepdims=True) * learning_rate

    # Print loss occasionally
    if epoch % 1000 == 0:
        loss = np.mean(np.square(error))
        print(f"Epoch {epoch} - Loss: {loss:.4f}")

# Final results
print("\nFinal predictions after training:")
print(final_output.round(3))

Epoch 0 - Loss: 0.3322
Epoch 1000 - Loss: 0.2490
Epoch 2000 - Loss: 0.2444
Epoch 3000 - Loss: 0.2180
Epoch 4000 - Loss: 0.1561
Epoch 5000 - Loss: 0.0560
Epoch 6000 - Loss: 0.0209
Epoch 7000 - Loss: 0.0109
Epoch 8000 - Loss: 0.0069
Epoch 9000 - Loss: 0.0049

Final predictions after training:
[[0.047]
 [0.957]
 [0.925]
 [0.072]]
